<a href="https://colab.research.google.com/github/Corne173/Corne173.github.io/blob/main/TradingBotV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Import modules





Install modules that are not in the colab environment

In [2]:
!pip install python-binance
!pip install discord-webhook

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 65 kB 2.0 MB/s 
     |████████████████████████████████| 292 kB 10.1 MB/s 
     |████████████████████████████████| 106 kB 58.1 MB/s 
     |████████████████████████████████| 45 kB 3.3 MB/s 
     |████████████████████████████████| 749 kB 48.1 MB/s 
  Attempting uninstall: regex
    Found existing installation: regex 2022.6.2
    Uninstalling regex-2022.6.2:
      Successfully uninstalled regex-2022.6.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from datetime import datetime
from binance import Client
import traceback
import numpy as np
import pandas as pd 
import json
from time import ctime,sleep
import csv
from discord_webhook import DiscordWebhook
from os.path import exists

with open('logging.csv', 'a') as file:
	writer = csv.writer(file)
	data = [ctime(),"Bot started"] 
	writer.writerow(data)



In [4]:
ctime()

'Sun Nov  6 20:13:14 2022'

# 2. Initialise variables

Define API keys ect here

In [5]:
#api_key = 'mkkQWNVZeqjP3HX4TygsV5j2cvp4RmyBMiBgVM8F3qzeeTlFdkldRhvqqYDz2XzA'
#api_secret = 'KYdu4TXphLffRAW1w3DNufMg6UyVnAQq0SBGxbSPI2ekCFynv7XYPPmIabq5fz9M'
#client = Client(api_key, api_secret)

#Evan's key 
#Y26RmiAgTj891sR8ZWqPrKI8XlF4yu0LZWMHFv49dkAusalLlXH8DbTh0ravuQSI
api_key = 'Y26RmiAgTj891sR8ZWqPrKI8XlF4yu0LZWMHFv49dkAusalLlXH8DbTh0ravuQSI'
api_secret = 'RHDs8Q3QBqswHPRxSYv0xmXgZfPmAiDyQQGcrXsEazzDTdIanR89iqEAxllVoTDi'
client = Client(api_key, api_secret)

Define bot parameters

In [6]:
curr = "AVAXBUSD"
coin = 'AVAX'
stable = 'BUSD'
timeframe = '60' #1h
timeunit = 'h' #m - minute.... h for hour

TSL_FACTOR = 0.01
TP_FACTOR = 0.0025
TP_Offset = 0.02
QtyAmt = 0.95

# SMA 
FastEMA = 3
MedEMA = 6
SlowSMA= 10
intervalSize = "1h"
numberOfKlines = 20 

ATRmult = 0.0075
SlPoints = 3
SlOffset = 1
ATRvalue = 14

#Bullish/Bearish Pin bar tolerance:
n1 = 0.51

# discord link
info_discord_webhook_URL = "https://discord.com/api/webhooks/1038515017043148901/p6tQl8V0JZurIt9trXKvvoPk-wdpHM5WtX7fP58fbUffVP0FTEd-WIMzXn-AYFCNcfji"
error_discord_webhook_URL  = "https://discord.com/api/webhooks/1038515683199295548/9p-aC_ubrhYWWQ5CSPb2EkwMUW8YSKuFuTInyBhDMgdtRgyKjbVjV_RjczRdInWugG03"
general_discord_webhook_URL = "https://discord.com/api/webhooks/1036171462488375307/CIqRxJjJ1FHqIK7zP98rBC7mg_ROwHXPOY3VVocMEuAWAuotzJJm4Ibv0wS14XfusTj4"

Create a test DataFrame

In [7]:
prices = client.get_all_tickers()
len(prices)

2123

In [8]:
#check to see if data already exists, load it if it does
if exists("position.csv"):
  posframe = pd.read_csv("position.csv",index_col=0)

else: # creates the DF
 # posframe = pd.DataFrame({
 #     "currency":["AVAXBUSD"],
 #     "qty":[1],
 #     "in position":[1],
 #     "long":[1],
 #     "short":[0],
 #     'buyprice':[19.19],
 #     "benchmark":[19.19]
 # })

  posframe = pd.DataFrame({
      "currency":["AVAXBUSD"],
      "qty":[0],
      "in position":[0],
      "long":[0],
      "short":[0],
      'buyprice':[0],
      "benchmark":[0]
  })
  # create CSV
  posframe.to_csv("position.csv")

In [9]:
posframe

,currency,qty,in position,long,short,buyprice,benchmark
0,AVAXBUSD,0,0,0,0,0,0


In [10]:
def log(df,currency,entryExit, postitionType,qty,buyprice):
  with open('logging.csv', 'a') as file:
    writer = csv.writer(file)
    data = [ctime(),currency,entryExit, postitionType,qty,buyprice] 
    writer.writerow(data)


In [11]:
def to_Discord(msg,channel=None):
  if postToDiscord:
    if channel == "info":
      message = f"{ctime()}\t {msg}"
      DiscordWebhook(url=info_discord_webhook_URL, content=message).execute()
    elif channel== "error":
      message = f"{ctime()}\t {msg}"
      DiscordWebhook(url=error_discord_webhook_URL, content=message).execute()
    else:
      message = f"{ctime()}\t {msg}"
      DiscordWebhook(url=general_discord_webhook_URL, content=message).execute()

Assuming the current CSV layout is as follows:
Currency,position
AVAXBUSD, 0

-------- 

AMMEND TO:
Currency,position,long,short
AVAXBUSD, 0, 0, 0

# Test and debugging code

In [12]:
#posframe = changepos(posframe, currency = "TestCurrency",entryExit="entry", postitionType="short",qty=696,price=14.12)
#posframe

In [13]:
#posframe = changepos(posframe, currency = "TestCurrency",entryExit="exit", postitionType="short",price=10.12)
#posframe

# All the supporting functions come here

The method/function that gets the data, calcuates the indicators, executes the commands... basically everything

In [14]:
def getpricedata(curr, intvalSize,numKlines):
    frame = pd.DataFrame(client.get_historical_klines(curr,intvalSize,limit=numKlines))
    frame = frame.iloc[:,:5]
    frame.columns = ['Time','Open','High','Low','Close']
    frame[['Open','High','Low','Close']]= frame[['Open','High','Low','Close']].astype(float)
    frame.Time = pd.to_datetime(frame.Time, unit='ms')
    return frame

In [15]:
def takeProfit(posframe,spotprice,TP_FACTOR, TP_Offset):
  if posframe["long"].values[0]:
    buyprice = posframe["buyprice"].values[0]
    index = posframe[posframe['currency']==curr].index[0]
    benchmark = posframe.at[index,'benchmark']
    TP = buyprice * (1+ TP_FACTOR)

    #updating the benchmark
    if spotprice > benchmark:
      posframe.at[index,'benchmark'] = spotprice
      if posframe.at[index,'benchmark'] >= TP:
        TTP = (posframe.at[index,'benchmark'] - TP_Offset)


  if posframe["short"].values[0]:
    buyprice = posframe["buyprice"].values[0]
    index = posframe[posframe['currency']==curr].index[0]
    benchmark = posframe.at[index,'benchmark']
    TP = buyprice * (1-TP_FACTOR)
    #updating the benchmark
    if spotprice < benchmark:
      posframe.at[index,'benchmark'] = spotprice
      if posframe.at[index,'benchmark'] <= TP:
        TTP = (posframe.at[index,'benchmark'] + TP_Offset)

    return TTP

In [16]:
class Trader(): #All information that belongs to a certain currency is saved in this class. Makes life much simpler

  def __init__(self, currency):
        self.currency = currency
        # make sure to get only for this currency
        posframe = pd.read_csv("position.csv",index_col=0).values[0]
        print(posframe)
        self.posframe = {
            "currency":posframe[0],
            "qty":posframe[1],
            "in position":posframe[2],
            "long":posframe[3],
            "short":posframe[4],
            'buyprice':posframe[5],
            "benchmark":posframe[6]
        }
        self.TSL_FACTOR = TSL_FACTOR
        self.longQTY = 1
        self.shortQTY = 1
        print(self.posframe)


  def run(self):
    self.indicators()
    print(self.posframe)
    if not self.posframe["in position"]:
      print(f"\rNOT in a position for {self.currency} ", end="")
      if self.entryLongConditions():
        self.enterLong()
      elif self.entryShortConditions():
        self.enterShort()
   
    elif self.posframe["in position"]:
      print(f"\rIN a position for {self.currency} ", end="")
      if self.posframe["long"]:   
        if self.exitLongConditions():
          self.exitLong()
      elif self.posframe["short"]:
        if self.exitShortConditions():
          self.exitShort()
    else:
      #do nothing
      print(f"\rNot a valid position ", end="")


# --------------- Under the hood -------------------------------------------------------

  def indicators(self):
    df = getpricedata(self.currency,intervalSize,numberOfKlines)
    #print(df)
    df['FastEMA'] = df['Close'].ewm(span=FastEMA, adjust=False).mean()
    df['MedEMA']= df['Close'].ewm(span=MedEMA, adjust=False).mean()
    df['SlowSMA'] = df.Close.rolling(SlowSMA).mean()
    high_low = df['High'] - df['Low']
    high_close = np.abs(df['High'] - df['Close'].shift())
    low_close = np.abs(df['Low'] - df['Close'].shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)
    df['ATR'] = true_range.rolling(ATRvalue).sum()/ATRvalue
    df['bullishPinBar'] = ((df.Close > df.Open) & ((df.Open - df.Low)   > (n1 * (df.High - df.Low)))) | ((df.Close < df.Open) & ((df.Close - df.Low) > (n1 * (df.High - df.Low))))
    df['bearishPinBar'] = ((df.Close > df.Open) & ((df.High - df.Close) > (n1 * (df.High - df.Low)))  | ((df.Close < df.Open) & ((df.High - df.Open) > (n1 * (df.High - df.Low)))))
    df['UpTrend'] = (df.FastEMA > df.MedEMA) & (df.MedEMA > df.SlowSMA)
    df['DnTrend'] = (df.FastEMA < df.MedEMA) & (df.MedEMA < df.SlowSMA)
    df['bullPierce'] = ((df.Low < df.FastEMA)  & (df.Open > df.FastEMA) & (df.Close > df.FastEMA)) | ((df.Low  < df.MedEMA) & (df.Open > df.MedEMA) & (df.Close > df.MedEMA)) | ((df.Low < df.SlowSMA)  & (df.Open > df.SlowSMA) & (df.Close > df.SlowSMA)) |((df.Low < df.FastEMA) & (df.Open < df.FastEMA) & (df.Close > df.FastEMA))
    df['bearPierce'] = ((df.High > df.FastEMA) & (df.Open < df.FastEMA) & (df.Close < df.FastEMA)) | ((df.High > df.MedEMA) & (df.Open < df.MedEMA) & (df.Close < df.MedEMA)) | ((df.High > df.SlowSMA) & (df.Open < df.SlowSMA) & (df.Close < df.SlowSMA)) |((df.Low > df.FastEMA) & (df.Open > df.FastEMA) & (df.Close < df.FastEMA))
    df.dropna(inplace=True)
    #print(df)
    self.df = df
    self.lastrow = df.iloc[-1]
    self.secondlastrow = df.iloc[-2]
    self.spot_price = self.lastrow.Close
    self.trailingStopLoss()
    self.coins = float(client.get_asset_balance(asset='AVAX')["free"])
    self.stableCoins = float(client.get_asset_balance(asset='USDT')["free"])


# -------------------    Check for position change signals-----------------------------
                    # ENTRY LONG
  def entryLongConditions(self):
    # comment out which ever condition is not wanted
    conditions = [
                  self.lastrow.UpTrend, 
                  self.secondlastrow.bullishPinBar , 
                  self.lastrow.bullPierce , 
                  not self.secondlastrow.bearishPinBar
                  ]
    signal = all(conditions)
    print(f"looking for entryLongConditions: {conditions} ", end="")
    return signal

                  # EXIT LONG
  def exitLongConditions(self):
    # comment out which ever condition is not wanted
    conditions = [
                  self.lastrow.FastEMA < self.lastrow.MedEMA, 
                  self.lastrow.Close < self.TSL,
                  #self.lastrow.Close < self.TTP,
                  ]
    signal = any(conditions)
    print(f"looking for exitLongConditions: {conditions} , TSL: {self.TSL}, ", end="")
    return signal

                # ENTRY SHORT
  def entryShortConditions(self):
    conditions = [
                  self.lastrow.DnTrend,
                  self.secondlastrow.bearishPinBar ,
                  self.lastrow.bearPierce,
                  not self.secondlastrow.bullishPinBar,
                 ]
    signal = all(conditions)
    print(f"looking for entryShortConditions: {conditions} ", end="")
    return signal

                  # EXIT SHORT
  def exitShortConditions(self):
    conditions = [
                  self.lastrow.FastEMA > self.lastrow.MedEMA ,
                  self.lastrow.Close > self.TSL,
                  #self.lastrow.Close > self.TTP,
                 ]
    signal = all(conditions)
    print(f"looking for exitShortConditions: {conditions}, TSL: {self.TSL} ") 
    return signal

# ------------------------------ Change position ---------------------------------------------------------
  def enterLong(self):
    print("Entering LONG position")
    if (self.spot_price/self.stableCoins < 1 ):
      to_Discord(f"Insuffient funds buy long {curr} ")
      exit()
    else:
      try:
        order = client.order_market_buy(symbol=self.currency, quantity=self.longQTY)
        buyprice = float(order['fills'][0]['price']) # used for performance metrics - has future use
        qtybought = float(order['fills'][0]['qty'])
        self.changepos(entryExit = "entry", postitionType = "long" ,qty = qtybought, price = buyprice)
      except:
        error_msg = traceback.format_exc()
        print(error_msg)
        to_Discord(error_msg,"error")

  def exitLong(self):
    print("Exiting LONG position")
    try:
      order = client.order_market_sell(symbol=self.currency, quantity=self.posframe["qty"]) 
      self.changepos(entryExit = "exit", postitionType = "long" , price = self.spot_price)
    except:
      error_msg = traceback.format_exc()
      print(error_msg)
      to_Discord(error_msg,"error")

  def enterShort(self):
    print("Entering SHORT position")
    try:
      order = client.order_market_sell(symbol=self.currency,quantity=self.shortQTY)
      buyprice = float(order['fills'][0]['price']) # used for performance metrics - has future use
      qtybought = float(order['fills'][0]['qty'])
      self.changepos(entryExit = "entry", postitionType = "short" ,qty = qtybought, price = buyprice)
    except:
      error_msg = traceback.format_exc()
      print(error_msg)
      to_Discord(error_msg,"error")

  def exitShort(self):
    print("Exiting LONG position")
    try:
      order = client.order_market_buy(symbol=self.currency, quantity = self.posframe["qty"]) 
      print(order)
      self.changepos(entryExit = "exit", postitionType = "short" , price = self.spot_price)
    except:
      error_msg = traceback.format_exc()
      print(error_msg)
      to_Discord(error_msg,"error")

# ------------------------End of change position ------------------------------------------------------

  #Trailing Stop Loss for LOng
  def trailingStopLoss(self):
    # the trailing stop loss ratchets behind the spot price. Everytime a new high or low is set, the TLS is moved accordingly
    if self.posframe["long"]:
      if self.spot_price > self.posframe['benchmark']:
        self.posframe['benchmark'] = self.spot_price
      self.TSL = (1-TSL_FACTOR)*self.posframe['benchmark']

    if self.posframe["short"]:
      if self.spot_price < self.posframe['benchmark']:
        self.posframe['benchmark'] = self.spot_price
      self.TSL = (1+TSL_FACTOR)*self.posframe['benchmark']
      

  def changepos(self,entryExit, postitionType, price,qty=None):
    #log(df,currency,entryExit, postitionType,qty,price)
  
    msg = ctime()
    msg = msg + f"\t {entryExit} {postitionType} position for {self.currency}. "
    
    if entryExit == "entry":
      msg = msg + f"QTY: {qty} @ spot price of {price}"
      #add to the dataframe
      if postitionType == "long":
        
            self.posframe = {
            "currency":self.currency,
            "qty":qty,
            "in position":1,
            "long":1,
            "short":0,
            "buyprice":price,
            "benchmark":price   #  used by the TSL. If it falls below % of the max, exit
        }
          
      if postitionType == "short":
        self.posframe = {
            "currency":self.currency,
            "qty":qty,
            "in position":1,
            "long":0,
            "short":1,
            "buyprice":price,
            "benchmark":price
        }
        
#------- end for entry positions -----------

    elif entryExit == "exit":
      # finds the index of the position which is then used to remove that row from the DF 
      
      if postitionType == "long":
        buyprice = self.posframe["buyprice"]
        msg = msg + f"Entered @ {buyprice} Exited @ {price}. Win/Loss of {round( (price-buyprice)/buyprice *100 , 3)}%" 
        self.posframe = {
            "currency":self.currency,
            "qty":0,
            "in position":0,
            "long":0,
            "short":0,
            "buyprice":0,
            "benchmark":0
        }

      
      if postitionType == "short":
        buyprice = self.posframe["buyprice"]
        msg = msg + f"Entered @ {buyprice} Exited @ {price}. Win/Loss of {round( (buyprice-price)/buyprice *100 , 3)}%" 
        self.posframe = {
            "currency":self.currency,
            "qty":0,
            "in position":0,
            "long":0,
            "short":0,
            "buyprice":0,
            "benchmark":0
        }
        
    to_Discord(msg,"info")
    
    pd.DataFrame({
            "currency":[posframe["currency"]],
            "qty":[posframe["qty"]],
            "in position":[posframe["in position"]],
            "long":[posframe["long"]],
            "short":[posframe["short"]],
            "buyprice":[posframe["buyprice"]],
            "benchmark":[posframe["benchmark"]]
    }).to_csv('position.csv')

    #self.posframe.to_csv('position.csv')


# MAIN LOOP

Here is your while loop that you work with that will execture endlessly. The code will execute sequentially and loop back to the start. Use this to implement the code flow diagram. 

Nothing should come after this. All the test code and functions that will be used in this loop should be written above.


In [17]:
 print(curr,intervalSize,numberOfKlines)
 df = getpricedata(curr,intervalSize,numberOfKlines)
 

AVAXBUSD 1h 20


In [18]:
prices = client.get_all_tickers()
currenciesOwned = []
listOfCurrenciesOwned = []
data = client.get_account()
for asset in data["balances"]:
  if float(asset["free"]) >0:
    currenciesOwned.append(asset)
    listOfCurrenciesOwned.append(asset["asset"])
    print(asset)

currenciesOwned
stableCoins = ["BUSD","USDT"]
stableCoinOfChoice = "BUSD"

#print(currenciesOwned)
listOfCurrenciesOwned 
for coin in stableCoins:
  listOfCurrenciesOwned.remove(coin)

result = []
for curr in listOfCurrenciesOwned:
  result.append(curr + stableCoinOfChoice)
print(result)
listOfCurrenciesOwned = result


total = 0
for price in prices:
  if price["symbol"] in listOfCurrenciesOwned:
     index = listOfCurrenciesOwned.index(price["symbol"])

     total += float(price["price"])*float(currenciesOwned[index]["free"])
     print(price["symbol"],float(price["price"]) * float(currenciesOwned[index]["free"]))

print(f"Total: {total}")

{'asset': 'BTC', 'free': '0.00024578', 'locked': '0.00000000'}
{'asset': 'BNB', 'free': '0.00408713', 'locked': '0.00000000'}
{'asset': 'USDT', 'free': '9.25225800', 'locked': '0.00000000'}
{'asset': 'BUSD', 'free': '71.15000000', 'locked': '0.00000000'}
{'asset': 'LUNA', 'free': '2.54890402', 'locked': '0.00000000'}
{'asset': 'AVAX', 'free': '11.56588808', 'locked': '0.00000000'}
{'asset': 'LUNC', 'free': '555316.78000000', 'locked': '0.00000000'}
['BTCBUSD', 'BNBBUSD', 'LUNABUSD', 'AVAXBUSD', 'LUNCBUSD']
BNBBUSD 1.4427568899999998
BTCBUSD 5.209764250800001
LUNABUSD 22.508893262399997
AVAXBUSD 1346.8695
LUNCBUSD 0.0006175739550058001
Total: 1376.031531977155


In [19]:
#order = client.order_market_buy(symbol="AVAXBUSD",
#                                           quantity=0.1)
#order

In [ ]:
postToDiscord = True
to_Discord("TradingBot V2 has started","info")

avaxbusd = Trader("AVAXBUSD") # creates an instance of the class where all variables and properties belong to this currency


try:
    while True:
      avaxbusd.run()
      sleep(10)
except Exception:
  error_msg = traceback.format_exc()
  print(error_msg)
  to_Discord(error_msg)    
  quit()
      
      


['AVAXBUSD' 0 0 0 0 0 0]
{'currency': 'AVAXBUSD', 'qty': 0, 'in position': 0, 'long': 0, 'short': 0, 'buyprice': 0, 'benchmark': 0}
{'currency': 'AVAXBUSD', 'qty': 0, 'in position': 0, 'long': 0, 'short': 0, 'buyprice': 0, 'benchmark': 0}
NOT in a position for AVAXBUSD looking for entryLongConditions: [False, False, False, True] looking for entryShortConditions: [True, False, False, True] {'currency': 'AVAXBUSD', 'qty': 0, 'in position': 0, 'long': 0, 'short': 0, 'buyprice': 0, 'benchmark': 0}
NOT in a position for AVAXBUSD looking for entryLongConditions: [False, False, False, True] looking for entryShortConditions: [True, False, False, True] {'currency': 'AVAXBUSD', 'qty': 0, 'in position': 0, 'long': 0, 'short': 0, 'buyprice': 0, 'benchmark': 0}
NOT in a position for AVAXBUSD looking for entryLongConditions: [False, False, False, True] looking for entryShortConditions: [True, False, False, True] {'currency': 'AVAXBUSD', 'qty': 0, 'in position': 0, 'long': 0, 'short': 0, 'buyprice': 